# Advanced Lane Lines Project

This is an improvement for project #1 where we just maked the lane lines, but without caring for the radius or the relative position of the car.

The goals / steps of this project are the following:

- Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
- Apply a distortion correction to raw images.
- Use color transforms, gradients, etc., to create a thresholded binary image.
- Apply a perspective transform to rectify binary image ("birds-eye view").
- Detect lane pixels and fit to find the lane boundary.
- Determine the curvature of the lane and vehicle position with respect to center.
- Warp the detected lane boundaries back onto the original image.
- Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle
position.

### Camera Calibration
1. Have the camera matrix and distortion coefficients been computed correctly and
checked on one of the calibration images as a test?

In [1]:
## Get all the needed imports

import matplotlib.pyplot as plt
import cv2
from glob import glob
import matplotlib.image as mpimage
import numpy as np

### Pipeline (single images)

#### Distortion correction.
Has the distortion correction been correctly applied to each image?

#### Color transformation and gradients
Has a binary image been created using color transforms, gradients or other methods?

Has a perspective transform been applied to rectify the image?

Have lane line pixels been identified in the rectified image and fit with a polynomial?

Having identified the lane lines, has the radius of curvature of the road been estimated?
And the position of the vehicle with respect to center in the lane?

### Pipeline (video)

## Discussion